In [1]:
import pandas as pd
import numpy as np

# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

In [2]:
routes

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178.0,WYA,6334.0,ADL,3341.0,NaN,0,SF3
67659,ZM,19016.0,DME,4029.0,FRU,2912.0,NaN,0,734
67660,ZM,19016.0,FRU,2912.0,DME,4029.0,NaN,0,734
67661,ZM,19016.0,FRU,2912.0,OSS,2913.0,NaN,0,734


In [4]:
route_sub = routes[["source","dest"]]
cUnique = set()
cUnique.add("LEX")
lexroutes = routes[routes["source"]=="LEX"]
lexroutes = routes[routes["source"]=="LEX"].drop_duplicates(subset=["source","dest"])[["source","dest"]]
clist = lexroutes["dest"].to_list()
cUnique = cUnique.union(clist)

In [5]:
lexroutes

,source,dest
3588,LEX,ATL
5763,LEX,CLT
5764,LEX,DFW
5765,LEX,ORD
21096,LEX,DCA
21097,LEX,DTW
21098,LEX,LGA
21099,LEX,MSP
29047,LEX,FLL
29048,LEX,PGD


In [8]:
cUnique

{'ATL',
 'CLT',
 'DCA',
 'DFW',
 'DTW',
 'FLL',
 'IAH',
 'LEX',
 'LGA',
 'MSP',
 'ORD',
 'PGD',
 'PIE',
 'SFB'}

In [19]:
def unique_func(df,cUnique):
    for row in df.iterrows():
        if row["dest_1"] in cUnique:
            bValue = True
        else:
            bValue = False
    return bValue
    


In [23]:
temp = pd.merge(lexroutes,route_sub,left_on="dest",right_on="source", how="inner",suffixes=('_org', '_1'))
temp["drop"]=temp["dest_1"].apply(lambda x: x in cUnique)
temp[["source_org","source_1","dest_1","drop"]]

,source_org,source_1,dest_1,drop
0,LEX,ATL,LWB,False
1,LEX,ATL,MCN,False
2,LEX,ATL,MEI,False
3,LEX,ATL,MSL,False
4,LEX,ATL,PIB,False
...,...,...,...,...
3553,LEX,IAH,PHX,False
3554,LEX,IAH,SAL,False
3555,LEX,IAH,ATL,True
3556,LEX,IAH,MTY,False


In [3]:
# get the first transfer list and connections
xfer1 = pd.merge(lexroutes,route_sub,left_on="dest",right_on="source", how="inner",suffixes=('_org', '_1'))
xfer1["drop"] = xfer1["dest_1"].apply(lambda x: x in cUnique)
xfer1 = xfer1[xfer1["drop"]==True]
clist = xfer1["dest_1"].to_list()
cUnique = cUnique.union(clist)
xfer1.drop(['drop'], axis=1, inplace=True)
xfer1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 402 entries, 16 to 3557
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source_org  402 non-null    object
 1   dest_org    402 non-null    object
 2   source_1    402 non-null    object
 3   dest_1      402 non-null    object
dtypes: object(4)
memory usage: 15.7+ KB


In [9]:
# get the second transfer list and connections
xfer2 = pd.merge(xfer1,route_sub,left_on="dest_1",right_on="source", how="inner")
xfer2["drop"] = xfer2["dest"].apply(lambda x: x in cUnique)
xfer2 = xfer2[xfer2["drop"]==True]
clist = xfer2["dest"].to_list()
cUnique = cUnique.union(clist)
xfer2.drop(['drop'], axis=1, inplace=True)
xfer2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18690 entries, 0 to 168708
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source_org  18690 non-null  object
 1   dest_org    18690 non-null  object
 2   source_1    18690 non-null  object
 3   dest_1      18690 non-null  object
 4   source      18690 non-null  object
 5   dest        18690 non-null  object
dtypes: object(6)
memory usage: 1022.1+ KB


In [13]:
xfer_final = xfer2[["source","dest"]]
xfer_final.drop_duplicates()

,source,dest
0,LEX,ATL
1,LEX,CLT
2,LEX,DFW
3,LEX,ORD
6,LEX,DCA
...,...,...
91922,ATL,DCA
91927,ATL,DTW
91938,ATL,FLL
91955,ATL,IAH


In [18]:
# Select the LEX routes, then join the source airports
lex_routes = xfer_final[(xfer_final['source']=="LEX")]
lex_routes = pd.merge(xfer_final, airports, left_on='source', right_on='iata', how='left')

# join the destination airports.  Here we need to use the suffixes option, because 
# the column names overlap, and we want to distinguish between source and dest
lex_routes = pd.merge(lex_routes, airports, 
                      left_on='dest', 
                      right_on='iata', 
                      how='left', 
                      suffixes=['_source','_dest'])

# here is what our data looks like
# lex_routes

In [19]:
# It looks like source has some duplicate names.  Drop the values from the airports
# file ane keep the one from the routes file
lex_routes = lex_routes.drop(['source_y','source'], axis=1)
lex_routes = lex_routes.rename(columns={'source_x': 'source'})

In [32]:
lunique = lex_routes["source"].unique
lunique

<bound method Series.unique of 0        LEX
1        LEX
2        LEX
3        LEX
6        LEX
        ... 
11136    ATL
11138    ATL
11139    ATL
11140    ATL
11142    ATL
Name: source, Length: 113, dtype: object>

In [29]:
# Let's keep only one route between each airport pair
# so we don't have a bunch of lines on top of each other
# The subset option tells it to consider just those columns when determining
# what is a duplicate. 

lex_routes = lex_routes.drop_duplicates(subset=['source', 'dest'])
lex_routes.head(50)

,source,dest,id_source,name_source,city_source,country_source,iata_source,icao_source,latitude_source,longitude_source,...,country_dest,iata_dest,icao_dest,latitude_dest,longitude_dest,altitude_dest,timezone_dest,dst_dest,tz_dest,type_dest
0,LEX,ATL,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
1,LEX,CLT,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport
2,LEX,DFW,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,DFW,KDFW,32.896801,-97.038002,607,-6.0,A,America/Chicago,airport
3,LEX,ORD,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,ORD,KORD,41.978600,-87.904800,672,-6.0,A,America/Chicago,airport
6,LEX,DCA,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,DCA,KDCA,38.852100,-77.037697,15,-5.0,A,America/New_York,airport
7,LEX,DTW,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,DTW,KDTW,42.212399,-83.353401,645,-5.0,A,America/New_York,airport
8,LEX,LGA,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,LGA,KLGA,40.777199,-73.872597,21,-5.0,A,America/New_York,airport
9,LEX,MSP,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,MSP,KMSP,44.882000,-93.221802,841,-6.0,A,America/Chicago,airport
10,LEX,FLL,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,FLL,KFLL,26.072599,-80.152702,9,-5.0,A,America/New_York,airport
11,LEX,PGD,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,...,United States,PGD,KPGD,26.920200,-81.990501,26,-5.0,A,America/New_York,airport


In [24]:
import folium
# create a basic map, centered on Lexington
lex_air = folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=4
)

In [25]:
import pyproj

# when creating a function, it is good practice to define the API!
def getGreatCirclePoints(startlat, startlon, endlat, endlon): 
    """
    startlat - starting latitude 
    startlon - starting longitude 
    endlat   - ending latitude 
    endlon   - ending longitude 
    
    returns - a list of tuples, where each tuple is the lat-long for a point
              along the curve.  
    """
    # calculate distance between points
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(startlon, startlat, endlon, endlat)

    # calculate line string along path with segments <= 20 km
    lonlats = g.npts(startlon, startlat, endlon, endlat,
                     1 + int(dist / 20000))

    # the npts function uses lon-lat, while the folium functions use lat-lon
    # This sort of thing is maddening!  What happens is the lines don't show
    # up on the map and you don't know why.  Learn from my mistakes
    latlons = []
    for lon_lat in lonlats: 
        
        # this is how you get values out of a tuple
        (lon, lat) = lon_lat
        
        # add them to our list
        latlons.append((lat, lon)) 
    
    # npts doesn't include start/end points, so prepend/append them
    latlons.insert(0, (startlat, startlon))
    latlons.append((endlat, endlon))
    
    return latlons


In [26]:
# define the map in the same way, but use great circles for the lines

# Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in lex_routes.iterrows():
    
    if row['source'] not in airport_set: 
        popup_string = row['city_source'] + ' (' + row['source'] + ')'
        marker = folium.CircleMarker([row["latitude_source"], row["longitude_source"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['source'])
        
    if row['dest'] not in airport_set: 
        popup_string = row['city_dest'] + '(' + row['dest'] + ')'
        marker = folium.CircleMarker([row["latitude_dest"], row["longitude_dest"]], 
                                     color='MidnightBlue',
                                     fill_color='MidnightBlue', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['dest'])
    
    # PolyLine will accept a whole list of tuples, not just two
    if (row['source'],row['dest']) not in route_set:            
        popup_string = row['source'] + '-' + row['dest']       
        
        gc_points = getGreatCirclePoints(row["latitude_source"], 
                                         row["longitude_source"], 
                                         row["latitude_dest"], 
                                         row["longitude_dest"])
        
        line = folium.PolyLine(gc_points, weight=2, popup=popup_string)
        line.add_to(lex_air)
        route_set.add((row['source'],row['dest']))
        
lex_air   

In [17]:
lex_routes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18690 entries, 0 to 18689
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   source_x          18690 non-null  object 
 1   dest              18690 non-null  object 
 2   id_source         18690 non-null  int64  
 3   name_source       18690 non-null  object 
 4   city_source       18690 non-null  object 
 5   country_source    18690 non-null  object 
 6   iata_source       18690 non-null  object 
 7   icao_source       18690 non-null  object 
 8   latitude_source   18690 non-null  float64
 9   longitude_source  18690 non-null  float64
 10  altitude_source   18690 non-null  int64  
 11  timezone_source   18690 non-null  float64
 12  dst_source        18690 non-null  object 
 13  tz_source         18690 non-null  object 
 14  type_source       18690 non-null  object 
 15  source_y          18690 non-null  object 
 16  id_dest           18690 non-null  int64 